# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
%matplotlib inline
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
part1_file = "weather_df.csv"

types_df = pd.read_csv(part1_file)

vacation_data_df = types_df.drop(columns=["Unnamed: 0"])
vacation_data_df

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,mataura,100,NZ,1618277964,95,-46.1927,168.8643,55.00,3.00
1,albany,100,US,1618277821,75,42.6001,-73.9662,50.00,4.52
2,anar darreh,0,FI,1618277983,93,68.9060,27.0288,17.60,1.14
3,jamestown,90,US,1618277856,81,42.0970,-79.2353,50.00,6.91
4,ginir,100,ET,1618277983,71,7.1333,40.7000,58.46,5.61
...,...,...,...,...,...,...,...,...,...
547,chattanooga,75,US,1618278177,33,35.0456,-85.3097,73.40,2.86
548,makokou,92,GA,1618278251,96,0.5738,12.8642,70.20,1.10
549,champerico,8,GT,1618278252,94,14.3000,-91.9167,69.01,11.36
550,mukhen,100,RU,1618278253,97,48.1000,136.1000,35.80,7.67


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
gmaps.configure(api_key=g_key)

In [4]:
locations = vacation_data_df[["Lat", "Lng"]]

humidity = vacation_data_df["Humidity"].astype(float)

fig = gmaps.figure(center = [0,0], zoom_level = 2)

heat_layer = gmaps.heatmap_layer(locations, weights = humidity, 
                                 dissipating=False, max_intensity=100, 
                                 point_radius=1)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
new_vacation_df = pd.DataFrame(vacation_data_df, columns = ["City", "Max Temp", "Wind Speed", "Cloudiness"])

max_temp = (new_vacation_df["Max Temp"] <= 80) & (new_vacation_df["Max Temp"] >= 70)
wind_speed = new_vacation_df["Wind Speed"] < 10
cloudiness = new_vacation_df["Cloudiness"] == 0

new_vacation_df[max_temp & wind_speed & cloudiness]

,City,Max Temp,Wind Speed,Cloudiness
214,doha,78.80,3.44,0
292,nizwa,73.74,1.03,0
337,manicore,78.80,1.72,0


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
hotel_df = pd.DataFrame(vacation_data_df, columns=["City", "Country", "Lat", "Lng"])

hotel_df["Hotel Name"] = ""

hotel_df

,City,Country,Lat,Lng,Hotel Name
0,mataura,NZ,-46.1927,168.8643,
1,albany,US,42.6001,-73.9662,
2,anar darreh,FI,68.9060,27.0288,
3,jamestown,US,42.0970,-79.2353,
4,ginir,ET,7.1333,40.7000,
...,...,...,...,...,...
547,chattanooga,US,35.0456,-85.3097,
548,makokou,GA,0.5738,12.8642,
549,champerico,GT,14.3000,-91.9167,
550,mukhen,RU,48.1000,136.1000,


In [ ]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

params = {"type" : "hotel",
          "keyword" : "hotel",
          "radius" : 5000,
          "key" : g_key}

for index, row in hotel_df.iterrows():
    lat = row["Lat"]
    lng = row["Lng"]
    
    params["location"] = f"{lat},{lng}"

    hotel_name = requests.get(base_url, params=params)
    
    hotel_name = hotel_name.json()
    print(json.dumps(hotel_name, indent=4, sort_keys=True))
    
    try:
        hotel_df.loc[index, "Hotel Name"] = hotel_name["results"][0]["name"]
        
    except (KeyError, IndexError):
        
        print("MISSING RESULT. SKIPPING...")
        
hotel_df

{
    "html_attributions": [],
    "results": [
        {
            "business_status": "OPERATIONAL",
            "geometry": {
                "location": {
                    "lat": -46.19353110000001,
                    "lng": 168.8649695
                },
                "viewport": {
                    "northeast": {
                        "lat": -46.19233507010728,
                        "lng": 168.8665862298927
                    },
                    "southwest": {
                        "lat": -46.19503472989273,
                        "lng": 168.8638865701072
                    }
                }
            },
            "icon": "https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/bar-71.png",
            "name": "Falls Hotel",
            "opening_hours": {},
            "photos": [
                {
                    "height": 3120,
                    "html_attributions": [
                        "<a href=\"https://maps.google.com/maps/contrib/10

{
    "html_attributions": [],
    "results": [
        {
            "business_status": "OPERATIONAL",
            "geometry": {
                "location": {
                    "lat": 42.110095,
                    "lng": -79.2851245
                },
                "viewport": {
                    "northeast": {
                        "lat": 42.11147222989272,
                        "lng": -79.28182884999998
                    },
                    "southwest": {
                        "lat": 42.10877257010728,
                        "lng": -79.28622305
                    }
                }
            },
            "icon": "https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/lodging-71.png",
            "name": "Chautauqua Harbor Hotel",
            "photos": [
                {
                    "height": 1393,
                    "html_attributions": [
                        "<a href=\"https://maps.google.com/maps/contrib/106257334849997488868\">A Google U

{
    "html_attributions": [],
    "results": [
        {
            "business_status": "OPERATIONAL",
            "geometry": {
                "location": {
                    "lat": 7.137535,
                    "lng": 40.712731
                },
                "viewport": {
                    "northeast": {
                        "lat": 7.138868979892721,
                        "lng": 40.71404947989272
                    },
                    "southwest": {
                        "lat": 7.136169320107277,
                        "lng": 40.71134982010727
                    }
                }
            },
            "icon": "https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/lodging-71.png",
            "name": "Bekera hotel",
            "place_id": "ChIJVZvfjevNyxcRAfDS5bdUUpE",
            "plus_code": {
                "compound_code": "4PQ7+23 Ginir, Ethiopia",
                "global_code": "6HV24PQ7+23"
            },
            "rating": 5,
          

{
    "html_attributions": [],
    "results": [],
    "status": "ZERO_RESULTS"
}
MISSING RESULT. SKIPPING...
{
    "html_attributions": [],
    "results": [
        {
            "business_status": "OPERATIONAL",
            "geometry": {
                "location": {
                    "lat": 40.5782811,
                    "lng": -124.1529536
                },
                "viewport": {
                    "northeast": {
                        "lat": 40.57965082989272,
                        "lng": -124.1513838201073
                    },
                    "southwest": {
                        "lat": 40.57695117010728,
                        "lng": -124.1540834798927
                    }
                }
            },
            "icon": "https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/lodging-71.png",
            "name": "THE REDWOOD RIVERWALK HOTEL",
            "opening_hours": {
                "open_now": true
            },
            "photos": [
   

{
    "html_attributions": [],
    "results": [],
    "status": "ZERO_RESULTS"
}
MISSING RESULT. SKIPPING...
{
    "html_attributions": [],
    "results": [
        {
            "business_status": "OPERATIONAL",
            "geometry": {
                "location": {
                    "lat": 20.6846539,
                    "lng": -105.2391801
                },
                "viewport": {
                    "northeast": {
                        "lat": 20.68608697989272,
                        "lng": -105.2379150201073
                    },
                    "southwest": {
                        "lat": 20.68338732010728,
                        "lng": -105.2406146798927
                    }
                }
            },
            "icon": "https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/lodging-71.png",
            "name": "Container Inn Puerto Vallarta",
            "opening_hours": {
                "open_now": true
            },
            "photos": [
 

...

{
    "html_attributions": [],
    "results": [
        {
            "business_status": "OPERATIONAL",
            "geometry": {
                "location": {
                    "lat": -43.3337176,
                    "lng": -65.0530194
                },
                "viewport": {
                    "northeast": {
                        "lat": -43.33237422010727,
                        "lng": -65.05165357010728
                    },
                    "southwest": {
                        "lat": -43.33507387989272,
                        "lng": -65.05435322989271
                    }
                }
            },
            "icon": "https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/lodging-71.png",
            "name": "Casa de mar",
            "photos": [
                {
                    "height": 1836,
                    "html_attributions": [
                        "<a href=\"https://maps.google.com/maps/contrib/104354339493582560058\">Iona Evans</

{
    "html_attributions": [],
    "results": [
        {
            "business_status": "OPERATIONAL",
            "geometry": {
                "location": {
                    "lat": -6.1637984,
                    "lng": 12.352401
                },
                "viewport": {
                    "northeast": {
                        "lat": -6.162419070107278,
                        "lng": 12.35375007989272
                    },
                    "southwest": {
                        "lat": -6.165118729892722,
                        "lng": 12.35105042010728
                    }
                }
            },
            "icon": "https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/lodging-71.png",
            "name": "Kinwica Resort-Hotel (Soyo, Angola)",
            "photos": [
                {
                    "height": 4608,
                    "html_attributions": [
                        "<a href=\"https://maps.google.com/maps/contrib/11803703058486153

{
    "html_attributions": [],
    "next_page_token": "ATtYBwI75frTyznsa0x5xKdjclXGtc6pZRe3baXyXKDYYbxX5tga4aMIhvkqRarM5QcQ6GsXVjhPMZnkIsz7M0-pyQA2MHBwka0iTVuuc16e4P277gTT9s34iLoeBLq5CH7h_DZoWwWBs205JgVxd5EqzrXBtCmlUKj1mMOSLktO5THMUppuZVsj0e5o7ff3xmL8uonRPG5EoLWA6a_jvixh3ga6ZewHBTyB4PHlkSCWYJfuxVXb8CqCKQq9M_ToeJuEU4pGYS2wCa4I6lCyp2Q9dZneBuFB7CKgxQy3TTVCpCPp0UKWSs8Ci7-WGDq5LM-g9Qbh2grtJHBto6ndGP4THN-VqbQM8F2QQQzBCACwE4_yQ1CkHAZt-lCTdNgPNHkF5cY9VVDzsuRhuzWcvWAsz1ycsaLwSmPBuCSEyHKIpxvLQ479RUou__xxfb0q48M",
    "results": [
        {
            "business_status": "OPERATIONAL",
            "geometry": {
                "location": {
                    "lat": 50.1243565,
                    "lng": -120.7502127
                },
                "viewport": {
                    "northeast": {
                        "lat": 50.12584352989273,
                        "lng": -120.7490392701073
                    },
                    "southwest": {
                        "lat": 50.1231438

{
    "html_attributions": [],
    "next_page_token": "ATtYBwLG4P5cMD3G-nutedkf_aD1bbCKBk1hL4BUF5RZnTZ1-s_7UGQApqdpZJQ-pEUwqJ_jM5VaqP5vSuUbL0VoNrHY6H_58jWpebYLkG7HP8dNHqGBa9qJJ8AYhmoH8vRdlS7QR_SRRE8swjuGW6gyVk-b-u6Rny3SuyJtdLmTYw49QchAoATfotAx7LikDvzbbfF9_lyqvaZD2iT5HBsmymKYlBOnJFv0awQeRkkBG75dxgJBuu4jrkE6b8qTnpRin_ZY7SX5emmFLoMdXid_bnIH6qnXcjb8UB-3s0sN-K6IBHbRbzHbFIowAz6WVdV6OEdNpfRtqtlkcKi_Py1a_PPt_vrgKXznW0W7BIwmU_qf_2jBVcfgylWs6h5vXEbue6YeQCgRrzGTNDyUKQIIqD1up-tE6Qo2vnZ_IDso25lBEIWEAr9EF1M4WhLm_cg",
    "results": [
        {
            "business_status": "OPERATIONAL",
            "geometry": {
                "location": {
                    "lat": -30.002445,
                    "lng": -50.132759
                },
                "viewport": {
                    "northeast": {
                        "lat": -30.00107907010728,
                        "lng": -50.13145107010728
                    },
                    "southwest": {
                        "lat": -30.0037787

{
    "html_attributions": [],
    "results": [
        {
            "business_status": "OPERATIONAL",
            "geometry": {
                "location": {
                    "lat": -34.6442677,
                    "lng": -58.5948731
                },
                "viewport": {
                    "northeast": {
                        "lat": -34.64292012010728,
                        "lng": -58.59354057010727
                    },
                    "southwest": {
                        "lat": -34.64561977989272,
                        "lng": -58.59624022989271
                    }
                }
            },
            "icon": "https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/lodging-71.png",
            "name": "Hotel Haedo",
            "opening_hours": {
                "open_now": true
            },
            "photos": [
                {
                    "height": 2016,
                    "html_attributions": [
                        "<a h

In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [ ]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content = hotel_info)

fig.add_layer(markers)

# Display figure
fig